In [ ]:
# 문장간에 유사한 문장 찾기

In [ ]:
# 1단계: 필요한 데이터 다운로드
import nltk
nltk.download('movie_reviews')
nltk.download('punkt')

# 2단계: 데이터 탐색
from nltk.corpus import movie_reviews

# 데이터셋 크기 파악
print(f"전체 영화 리뷰 수: {len(movie_reviews.fileids())}")
print(f"카테고리: {movie_reviews.categories()}")  # ['neg', 'pos']
print(f"부정 리뷰: {len(movie_reviews.fileids(categories='neg'))}개")
print(f"긍정 리뷰: {len(movie_reviews.fileids(categories='pos'))}개")

# 3단계: 첫 번째 리뷰 살펴보기
first_review_id = movie_reviews.fileids()[0]
first_review = movie_reviews.raw(first_review_id)
print(f"\n첫 번째 리뷰 ID: {first_review_id}")
print(f"원문 일부:\n{first_review[:200]}")

# 4단계: 토큰화 결과 확인
sentences = movie_reviews.sents(first_review_id)  # 문장 단위 토큰화
words = movie_reviews.words(first_review_id)      # 단어 단위 토큰화

print(f"\n문장 토큰화 (첫 2개):")
for i, sent in enumerate(sentences[:2]):
    print(f"  {i+1}: {sent}")

print(f"\n단어 토큰화 (첫 20개): {words[:20]}")

In [ ]:
# RegressTokenizer : 정규표현식으로 정확한 토큰화
# stowords : 문법적 기능을 제거하고 단어에 집중
# 상위 N개 단어 선택 : 메모리 효율성과 노이즈 제거의 균형

In [ ]:
# BOW - 수동으로 벡터 생성
# 1단계: 모든 문서를 단어 리스트로 변환
documents = [list(movie_reviews.words(fileid)) 
             for fileid in movie_reviews.fileids()]

print(f"전체 문서 수: {len(documents)}")
print(f"첫 문서의 단어 수: {len(documents[0])}")
print(f"첫 문서의 첫 50개 단어:\n{documents[0][:50]}")

# 2단계: 전체 단어 빈도 계산 (불용어 제외 전)
word_count = {}
for doc in documents:
    for word in doc:
        word_count[word] = word_count.get(word, 0) + 1

# 상위 10개 빈도 단어 확인
sorted_words = sorted(word_count.items(), key=lambda x: x[1], reverse=True)
print("\n상위 10개 빈도 단어:")
for i, (word, count) in enumerate(sorted_words[:10], 1):
    print(f"  {i}. '{word}': {count}회")

# 3단계: 불용어 제거 후 처리
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

# 정규표현식으로 3글자 이상의 단어만 추출
tokenizer = RegexpTokenizer(r"[\w']{3,}")
# 영어 불용어 로드
english_stops = set(stopwords.words('english'))

# 모든 리뷰를 토큰화하고 불용어 제거
processed_documents = []
for fileid in movie_reviews.fileids():
    raw_text = movie_reviews.raw(fileid)
    tokens = [token for token in tokenizer.tokenize(raw_text) 
              if token not in english_stops]
    processed_documents.append(tokens)

# 처리 후 단어 빈도 재계산
word_count_processed = {}
for doc in processed_documents:
    for word in doc:
        word_count_processed[word] = word_count_processed.get(word, 0) + 1

sorted_processed = sorted(word_count_processed.items(), 
                         key=lambda x: x[1], reverse=True)

print(f"\n전체 서로 다른 단어 수: {len(sorted_processed)}")
print("\n처리 후 상위 10개 단어:")
for i, (word, count) in enumerate(sorted_processed[:10], 1):
    print(f"  {i}. '{word}': {count}회")

# 4단계: 특성 선택 (상위 1000개 단어)
word_features = [word for word, count in sorted_processed[:1000]]
print(f"\n특성으로 선택된 단어 수: {len(word_features)}")
print(f"특성 예시: {word_features[:20]}")

In [ ]:
def document_features(document, word_features):
    """
    문서를 특성 벡터로 변환
    
    Args:
        document: 토큰화된 단어 리스트
        word_features: 특성으로 사용할 단어 리스트
    
    Returns:
        document의 각 특성에 대한 빈도 리스트
    """
    # 문서 내 단어 빈도 계산
    word_count = {}
    for word in document:
        word_count[word] = word_count.get(word, 0) + 1
    
    # 특성 벡터 생성
    features = []
    for word in word_features:
        # 특성 단어가 문서에 없으면 0
        features.append(word_count.get(word, 0))
    
    return features

# 테스트 실행
test_features = ['one', 'two', 'teen', 'couples', 'solo']
test_doc = ['two', 'two', 'couples']
result = document_features(test_doc, test_features)

print("테스트 단어 리스트:", test_features)
print("테스트 문서:", test_doc)
print("결과 벡터:", result)
print("→ 'two'가 2번, 'couples'가 1번, 나머지는 0")

# 모든 문서에 대해 특성 벡터 생성
feature_sets = [document_features(doc, word_features) 
                 for doc in processed_documents]

print(f"\n생성된 특성 벡터 수: {len(feature_sets)}")
print(f"각 벡터의 차원: {len(feature_sets[0])}")
print(f"\n첫 문서 벡터 (처음 20개):")
for i, (word, count) in enumerate(zip(word_features[:20], feature_sets[0][:20])):
    print(f"  '{word}': {count}")

In [ ]:
# CountVectorizer가 위절차를 수행한다.
# CountVectorizer 사용방법

In [ ]:
# processed_documents[0][:5] # 문장을 토큰화(3개의 연속된 문장, 불용어 제거)
for doc in processed_documents:
    english_stops

In [ ]:
# 각 문서의 고정된 길이의 벡터로 변환(모든 문서가 같은 차원)
# 기계학습 알고리즘의 입력형식으로 변환
def document_features(document, word_features):
    """
    
    """

###

In [ ]:
import pandas as pd
url = "C:\\Users\\Playdata2\\Downloads\\daum_movie_review.csv"
df = pd.read_csv(url)

In [ ]:
sample_review = df.review[1]
sample_review

In [ ]:
from konlpy.tag import Okt
okt = Okt()

In [ ]:
# 명사만 추출
okt.nouns(sample_review)

In [ ]:
okt.pos(sample_review)

In [ ]:
# 명사 동사 형용사 Noun Verb Adjective
[word for word, tag in okt.pos(sample_review) if tag in ['Noun','Verb','Adjective']]

In [ ]:
# 커스텀 토크나이저 함수로 생성
def custom_tokenizer(doc):
    return [word for word, tag in okt.pos(doc) if tag in ['Noun','Verb','Adjective']]
# 커스텀 토크나이저 함수를 만든 이유
# CountVectorizer에 커스텀 토크나이저 적용하기위해서, 그니깐 간략하게 적을려고

In [ ]:
# CountVectorizer에 커스텀 토크나이저 적용
from sklearn.feature_extraction.text import CountVectorizer
daum_cv = CountVectorizer(max_features=1000, tokenizer=custom_tokenizer)
daum_dtm = daum_cv.fit_transform(df.review)
daum_dtm.shape

In [ ]:
# 코사인 유사도 -1 ~ 1 사이의 값을 벡터간 각도기반 유사도 계산
# 0 에 가까운 값 : 직교(무관) 1에 가까우면 같은 방향
# 모든 유사도를 계산 : 상위 N개 결과 도출(추천 시스템의 기초) - 유튜브 알고리즘과 연관?

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
reviews = df.review.copy()
original_review = reviews[1]
words = original_review.split()

# 뒤 절반을 query 사용
midpoint = len(words) // 2
query_text = " ".join(words[midpoint:])
# 벡터로 변환
query_vector = daum_cv.transform([query_text])
query_vector

In [ ]:
reviews[1]

In [ ]:
# 모든 리뷰와 유사도 계산
# 첫번째 중간에서부터 뒷부분까지와 전체 데이터의 유사도를 비교하였다.
import numpy as np
similarity_scores = cosine_similarity(query_vector,daum_dtm)
most_similar_idx = np.argmax(similarity_scores)

# 가장 높은 유사도 값
highest_score = similarity_scores[0, most_similar_idx]

# 퍼센트로 변환
similarity_percent = highest_score * 100

print(f"가장 유사한 리뷰 인덱스: {most_similar_idx}")
print(f"유사도: {similarity_percent:.2f}%")
# 가장 유사한 리뷰  인덱스 가 첫번째 문장, 첫번째 문장 이 가장유사하다고 뜬....
# 본인을 제외한 그 이외의 상위 5개의 문장을 조회해보고자한다

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# query_vector: 쿼리 텍스트 벡터
# daum_dtm: 전체 리뷰 DTM
# daum_reviews: 전체 리뷰 리스트 또는 시리즈

# 1. 모든 리뷰와 유사도 계산
similarity_scores = cosine_similarity(query_vector, daum_dtm)[0]  # 1차원 배열

# 2. 자기 자신(가장 높은 유사도) 제외
most_similar_idx = np.argmax(similarity_scores)
similarity_scores[most_similar_idx] = 0

# 3. 상위 5개의 인덱스 추출
top5_idx = np.argsort(similarity_scores)[-5:][::-1]  # 내림차순
top5_scores = similarity_scores[top5_idx] * 100  # 퍼센트로 변환

# 4. 출력
print("원래 쿼리 텍스트:")
print(query_text)
print("\n본인을 제외한 상위 5개의 유사한 리뷰:")

for idx, score in zip(top5_idx, top5_scores):
    print(f"\n리뷰 인덱스: {idx}, 유사도: {score:.2f}%")
    print(reviews[idx])

In [ ]:
from konlpy.tag import Okt
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
url = "https://drive.google.com/uc?id=1KOKgZ4qCg49bgj1QNTwk1Vd29soeB27o"
df = pd.read_csv(url)
okt = Okt()
def custom_tokenizer(doc):
    """
    형태소 분석 후 명사, 동사, 형용사만 추출
    """
    pos_tags = okt.pos(doc)
    tokens = [word for word, pos in pos_tags 
              if pos in ['Noun', 'Verb', 'Adjective']]
    return tokens

daum_cv = CountVectorizer(
    max_features=1000,
    tokenizer=custom_tokenizer
)

reviews = df.review
daum_dtm  = daum_cv.fit_transform(reviews)
original_review = reviews[0]  # 첫 번째 리뷰
print(f"원본 리뷰 (처음 200자):\n{original_review[:200]}\n")

# 문서의 뒤 절반을 query로 사용 (부분 검색 시나리오)
midpoint = len(original_review) // 2
# query_text = original_review[midpoint:]  # 뒤 절반
query_text = original_review
print(f"쿼리 텍스트 (처음 150자):\n{query_text[:150]}\n")

# 2단계: 쿼리 문서를 벡터로 변환
query_vector = daum_cv.transform([query_text])
print(f"쿼리 벡터 크기: {query_vector.shape}")

# 유사도 분석
scores = cosine_similarity(query_vector,daum_dtm)
most_simular_idx = np.argmax(scores)
# scores[most_simular_idx], reviews[most_simular_idx]

In [ ]:
query_text

In [ ]:
np.array(reviews)[scores[0].argsort()[::-1][:5]]

In [ ]:
# 개선 TF-IDF
# 단어의 상대적 중요도를 반영한 벡터화 기법
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer
# TF : 특정단어가 문서에서 얼마나 자주나타나는 지 비율 (빈도)
# 해당 단어의 빈도 / 문서의 전체 단어 수 좋다 문서에서 10 번 해당 문서는 100단어 10 / 100
# IDF (Inverse Document Frequency) 단어가 전체 문장에서 얼마나 드문지 (희귀)
# log(전체 문서 / 해당 단어 포함 문서) 단어의 가중치를 낮추기 위해서 log적용
# 2000개 문서중 100개만 '좋다' log(2000/100) = 2.99
# TF-IDF TF x IDF --> 특정 문서에서 의미 있는 단어에 높은 가중치를 부여한다.

# 이 값이 클수록 문서내에서 단어 중요도가 높다 라고 판단하는 지표

In [ ]:
tfidf_cv = TfidfVectorizer( # 사용법은 CounterVectorizer와 유사
    max_features=1000,
    tokenizer=custom_tokenizer
)
tfidf_dtm = tfidf_cv.fit_transform(reviews) # 전체 문서를 TF-IDF 벡터화

count_dtm = daum_cv.fit_transform(reviews) # 전체 문서를 Bow 벡터화
count_dtm.shape, tfidf_dtm.shape

In [ ]:
# 쿼리 벡터화
query_count = daum_cv.transform([query_text])
query_tfidf = tfidf_cv.transform([query_text])

# 코사인 유사도 계산
count_sim = cosine_similarity(query_count,count_dtm)[0]
tfidf_sim = cosine_similarity(query_tfidf,tfidf_dtm)[0]

In [ ]:
a=np.array([1,2,3,4,5])
a.argsort() # 오름차순으로 인덱스
(-a).argsort() # 값에 -를 붙이면 .. 오름차순 인덱스는 내림차순과 같다

In [ ]:
# '돈 들인건 티가 나지만 보는 내내 하품만'

In [ ]:
# CountVectorizer	| TF-IDF
# 단어 출현 횟수	 | 단어 출현 × 희귀 단어 가중치

In [ ]:
top5_count_index = (-count_sim).argsort()[:5]
top5_tfidf_index = (-tfidf_sim).argsort()[:5]

In [ ]:

reviews = np.array(reviews)

In [ ]:
reviews[top5_count_index][1:]

In [ ]:
# 짧은 문장일수록 벡터의 크기가 작아,
# Cosine similarity 계산에서 “작게나마 비슷하다”고 판단될 가능성이 있음

In [ ]:
reviews[top5_tfidf_index][1:]

In [ ]:
# 실제 의미 유사성”과 상관없이 벡터 차원에서 겹치는 단어가 있거나,
# 희귀 단어 가중치 때문에 선택

In [ ]:

my_reivew = '숙면을 하기 좋은 영화.. 강추..'
# my_review  count 방식이나 또는 tf-idf 방식으로 벡터화 한후... 전체리뷰를 만약 tf-idf 방식이면 전체 리뷰를 tf-idf 벡터화 한
# 전체데이터와 함께 유사도 방식으로 점수를 구해서 상위 N개의 문서를 출력

In [ ]:
# 쿼리 벡터화
query_count = daum_cv.transform([my_reivew])
query_tfidf = tfidf_cv.transform([my_reivew ])

# 코사인 유사도 계산
count_sim = cosine_similarity(query_count,count_dtm)[0]
tfidf_sim = cosine_similarity(query_tfidf,tfidf_dtm)[0]

top5_count_index = (-count_sim).argsort()[:5]
top5_tfidf_index = (-tfidf_sim).argsort()[:5]


In [ ]:
# my_reivew = '숙면을 하기 좋은 영화.. 강추..'

In [ ]:

reviews[top5_count_index]

In [ ]:

reviews[top5_tfidf_index]

In [ ]:
# N-gram 연속된 N개의 단위
# unigram 1-gram
# 영화가 정말 재미있다
# [영화] [가] [정말] [재미있다]

# Bigram(2-gram)
# [영화 가] [가 정말] [정말 재미있다]
# 첫번째 단어와 마지막 단어는 한번씩 들어가겠네

# Trigram(3-gram)
# [영화 가 정말] [가 정말 재미있다]

# 특성
# 문맥정보 포함 : 단서의 순서와 관계를 반영
# 더 나은 분류 : 좋은 영화 나쁜 영화 구분
# 의미 보존 : 인접한 단어들의 의존성 보존


In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from konlpy.tag import Okt
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
df.head()

In [ ]:
df.rating.hist(bins=2) # 클래스 불균형 이 있어서 qcut을 사용

In [ ]:
# df['rating_group'] = pd.qcut(df['rating'], q=2, labels=[0, 1])

# # 히스토그램
# plt.hist(df['rating_group'], bins=2, edgecolor='black')
# plt.xticks([0, 1])
# plt.xlabel('Rating Group')
# plt.ylabel('Count')
# plt.show()
## df.drop('rating_group', axis=1, inplace=True)

In [130]:
df['label'] = (df.rating >= 7).astype(int)
df.tail(2)

,review,rating,date,title,label
14723,간만에 제대로 잘짜여진 각본의 영화를 봤네 여운이 아직도 남아~어른을 위한 애니~,10,2018.01.12,코코,1
14724,한국개봉을 눈빠지게 기다린 보람이있다 깨우치는게 많은 영화,10,2018.01.12,코코,1


In [131]:
def explain_ngrams(text, n_values=[1, 2, 3]):
    """N-gram을 이해하기 쉽게 설명"""
    tokens = okt.morphs(text)
    
    print(f"\n원본 텍스트: '{text}'")
    print(f"형태소 토큰: {tokens}\n")
    
    for n in n_values:
        ngrams = [' '.join(tokens[i:i+n]) for i in range(len(tokens) - n + 1)]
        print(f"{n}-gram ({['Unigram', 'Bigram', 'Trigram'][n-1]}):")
        print(f"  총 {len(ngrams)}개: {ngrams}")
        if len(ngrams) <= 10:
            for i, gram in enumerate(ngrams, 1):
                print(f"    {i}. [{gram}]")
        print()

# 샘플 텍스트에 대한 N-gram 설명
sample_review = "영화가 정말 재미있다"
explain_ngrams(sample_review)


원본 텍스트: '영화가 정말 재미있다'
형태소 토큰: ['영화', '가', '정말', '재미있다']

1-gram (Unigram):
  총 4개: ['영화', '가', '정말', '재미있다']
    1. [영화]
    2. [가]
    3. [정말]
    4. [재미있다]

2-gram (Bigram):
  총 3개: ['영화 가', '가 정말', '정말 재미있다']
    1. [영화 가]
    2. [가 정말]
    3. [정말 재미있다]

3-gram (Trigram):
  총 2개: ['영화 가 정말', '가 정말 재미있다']
    1. [영화 가 정말]
    2. [가 정말 재미있다]



In [137]:
# 벡터화 n-gram별 벡터화 및 특성 비교
def tokenizer_morphs(text):
    '''
    형태소 기반 토크나이저
    '''
    return okt.morphs(text)

# 1-gram
vec_1gram = TfidfVectorizer(tokenizer=tokenizer_morphs, ngram_range=(1,1), max_features=50)
X_1gram = vec_1gram.fit_transform(df.review)
print(f'차원 : {X_1gram.shape}')
features_1gram = vec_1gram.get_feature_names_out()[:20]
print(f'{','.join(features_1gram )}') # ^

차원 : (14725, 50)
!,,,.,..,...,?,cg,~,가,감동,것,고,과,그,너무,눈물,는,다,더,도


In [139]:
# 2-gram
vec_2gram = TfidfVectorizer(tokenizer=tokenizer_morphs, ngram_range=(1,2), max_features=50)
X_2gram = vec_2gram.fit_transform(df.review)
print(f'차원 : {X_2gram.shape}')
features_2gram = vec_2gram.get_feature_names_out()
bigram = [f for f in features_2gram if len(f.split()) > 1 ][:15]
print(f'{','.join(bigram)}')


차원 : (14725, 50)
들 이


In [140]:
# 데이터 분할
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(df.review,df.label,stratify=df.label,train_size=0.2,random_state=42)

In [141]:
# 1 gram 모델
from sklearn.linear_model import LinearRegression
vec_1 = TfidfVectorizer(tokenizer=custom_tokenizer,ngram_range=(1,1),max_features=1000)
x_train_1 = vec_1.fit_transform(x_train)
x_test_1 = vec_1.transform(x_test)

clf_1 = LogisticRegression()
clf_1.fit(x_train_1,y_train)
clf_1.score(x_test_1,y_test)

0.799151103565365

In [142]:
from sklearn.metrics import classification_report
predict_1 = clf_1.predict(x_test_1)
print( classification_report(y_test, predict_1) )

              precision    recall  f1-score   support

           0       0.81      0.38      0.51      3325
           1       0.80      0.97      0.87      8455

    accuracy                           0.80     11780
   macro avg       0.80      0.67      0.69     11780
weighted avg       0.80      0.80      0.77     11780

